In [6]:
import pandas as pd
import numpy as np 
import csv
import re
import string
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")

In [7]:
import pandas as pd
import numpy as np 
import csv
import re
import string
import emoji

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])
#demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
start = time.time()
path = '/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/h01-20200818-153021.csv'
data = pd.read_csv(path, index_col=None, header=0, engine='python' )
#data = pd.read_parquet(path)
end = time.time()
print("Read parquet with pandas: ",(end-start),"sec")

Read parquet with pandas:  0.7261269092559814 sec


In [7]:
data_en

,created_at,id,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,place_full_name,place_country_code,...,QT_username,QT_name,QT_location,QT_place_full_name,QT_place_country_code,QT_coordinates,QT_text,QT_full_text,RT_Flag,Date_Flag
0,Tue Aug 18 19:30:11 +0000 2020,1295805209722183684,False,NaN,NaN,NaN,NaN,en,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
1,Tue Aug 18 19:30:11 +0000 2020,1295805209545932802,True,1.295540e+18,NaN,NaN,NaN,en,NaN,NaN,...,JoeBiden,Joe Biden,"Wilmington, DE",NaN,NaN,NaN,".@AndrewCuomo, thank you for your leadership a...",".@AndrewCuomo, thank you for your leadership a...",True,True
2,Tue Aug 18 19:30:11 +0000 2020,1295805210040897536,True,1.295798e+18,NaN,NaN,NaN,en,NaN,NaN,...,ITROutlook,ITR Economics®,"Manchester, NH",NaN,NaN,NaN,“Our forecast for the US economy ... reflects ...,“Our forecast for the US economy ... reflects ...,False,True
3,Tue Aug 18 19:30:11 +0000 2020,1295805209923530763,True,1.295572e+18,NaN,NaN,NaN,en,NaN,NaN,...,LOUISAGUIRRE,Louis Aguirre,Los Angeles,NaN,NaN,NaN,Miami Dade’s antiquated storm drain system dum...,Miami Dade’s antiquated storm drain system dum...,False,True
4,Tue Aug 18 19:30:11 +0000 2020,1295805209634119681,True,1.295673e+18,NaN,NaN,NaN,en,NaN,NaN,...,barneyjiri,MANIFESTOR,High frequency,NaN,NaN,NaN,Gold Reef City would make the money they’ve l...,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,Tue Aug 18 19:34:16 +0000 2020,1295806236173238273,False,NaN,NaN,NaN,NaN,en,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
11225,Tue Aug 18 19:34:16 +0000 2020,1295806236416528384,False,NaN,NaN,NaN,NaN,en,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True
11226,Tue Aug 18 19:34:16 +0000 2020,1295806236349390850,False,NaN,NaN,NaN,NaN,en,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True
11227,Tue Aug 18 19:34:16 +0000 2020,1295806236504535041,False,NaN,NaN,NaN,NaN,en,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True


In [9]:
cols = [0, 1, 3, 4, 5,6, 7, 8, 14, 15, 16, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 41, 42, 43, 45, 51, 52, 53, 54, 55, 56, 58, 63, 64, 65, 67, 73, 74, 75, 76, 77]

data = data[data.columns.values[cols]]

data_en = data[data['lang'] == 'en'].reset_index(drop=True)

# RT flag

data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

# Date check

data_en['Date_Flag'] = data_en['created_at'].str[:10] == 'Tue Aug 18'

data_en = data_en[np.where((data_en['Date_Flag'] == True),True,False)].reset_index(drop=True)

# Tree

############### is_quote_tweet --> False --> Non RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

interim['extended_tweet_full_text'] = np.where(~interim['truncated'], interim['text'], interim['extended_tweet_full_text'])

non_rep = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> False --> RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == False) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> True --> Non RT ###############

interim = data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])

interim = interim[['created_at', 'id','user_id', 'coordinates', 'location', 'extended_tweet_full_text']]

interim.columns = non_rep.columns

interim = interim.drop_duplicates(['id']).reset_index(drop=True)

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

interim = interim[['created_at','QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

############### is_quote_tweet --> True --> RT ###############

interim = data_en[np.where((data_en['is_quote_tweet'] == True) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

interim = interim[['created_at', 'RT_id','RT_user_id', 'RT_coordinates', 'RT_location','RT_full_text']]

interim = interim.drop_duplicates(['RT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

interim = data_en[np.where((data_en['is_quote_tweet']== True ) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

interim['QT_full_text'] = np.where(interim['QT_full_text'].isna(),interim['QT_text'],interim['QT_full_text'])

interim = interim[['created_at', 'QT_id','QT_user_id', 'QT_coordinates', 'QT_location', 'QT_full_text']]

interim = interim.drop_duplicates(['QT_id']).reset_index(drop=True)

interim.columns = non_rep.columns

non_rep = non_rep.append(interim,ignore_index=True)

non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

In [10]:
non_rep

,created_at,id,user_id,coordinates,location,extended_tweet_full_text
0,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...
1,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...
2,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T..."
3,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...
4,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...
...,...,...,...,...,...,...
7766,Tue Aug 18 19:34:12 +0000 2020,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...
7767,Tue Aug 18 19:34:13 +0000 2020,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ..."
7768,Tue Aug 18 19:34:15 +0000 2020,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...
7769,Tue Aug 18 19:34:15 +0000 2020,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th..."


In [11]:
cleaned_data = non_rep.copy()

cleaned_data['extended_tweet_full_text_duplicate'] = cleaned_data['extended_tweet_full_text']

#Convert to lower case
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.lower()

#Removing emojis
def demoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text))

cleaned_data[u'extended_tweet_full_text'] = cleaned_data[u'extended_tweet_full_text'].astype(str)
cleaned_data[u'extended_tweet_full_text'] = cleaned_data[u'extended_tweet_full_text'].apply(lambda x:demoji(x))

#Remove URLs
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r"http\S+| www\S+| https\S+| \S+\.com\S+| \S+\.com", "", regex=True)

#Remove user @
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.replace(r'\@[\w]+', "", regex=True)

#Remove punctuations
cleaned_data['extended_tweet_full_text'] = cleaned_data['extended_tweet_full_text'].str.translate(str.maketrans("", "", string.punctuation))

#More Cleaning
cleaned_data['extended_tweet_full_text']=cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

#Tokenizing
cleaned_data.extended_tweet_full_text = cleaned_data.extended_tweet_full_text.astype(str)
cleaned_data['tokenized_extended_tweet_full_text'] = cleaned_data.apply(lambda row: nltk.word_tokenize(row.extended_tweet_full_text), axis=1)

# remove stopwords
cleaned_data['filtered_extended_tweet_full_text'] = cleaned_data['tokenized_extended_tweet_full_text'].apply(lambda x: [word for word in x if word not in stop_words])

#Stemming
ps = PorterStemmer()
cleaned_data['stemmed_extended_tweet_full_text'] = cleaned_data['filtered_extended_tweet_full_text'].apply(lambda x: [ps.stem(y) for y in x])

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#Lemmatizing
lemmatizer = WordNetLemmatizer()
cleaned_data['lemmatized_extended_tweet_full_text'] = cleaned_data['filtered_extended_tweet_full_text'].apply(lambda x: [lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in x])

#Joining the lemmetized tokens to form string
cleaned_data['final'] = cleaned_data['lemmatized_extended_tweet_full_text'].apply(lambda x: " ".join([word for word in x]))

#Remove punctuations
cleaned_data['final'] = cleaned_data['final'].str.translate(str.maketrans("", "", string.punctuation))
cleaned_data['final'] = cleaned_data['final'].str.replace("’", '').str.replace("“", '').str.replace("”", '')
cleaned_data['extended_tweet_full_text']=cleaned_data['extended_tweet_full_text'].astype(str).str.replace('/[^a-zA-Z0-9 ]/g', '', regex=True).str.replace('\n',' ', regex=True).str.replace('—',' ', regex=True).str.strip('“').str.strip('”').str.strip('’').str.lstrip(' ').str.rstrip(' ')

In [12]:
cleaned_data

,created_at,id,user_id,coordinates,location,extended_tweet_full_text,extended_tweet_full_text_duplicate,tokenized_extended_tweet_full_text,filtered_extended_tweet_full_text,stemmed_extended_tweet_full_text,lemmatized_extended_tweet_full_text,final
0,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,can we wear clone helmets instead of masks to ...,"[can, we, wear, clone, helmets, instead, of, m...","[wear, clone, helmets, instead, masks, to, sch...","[wear, clone, helmet, instead, mask, to, school]","[wear, clone, helmet, instead, mask, to, school]",wear clone helmet instead mask to school
1,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",sc senate to reconvene in sept to discuss covi...,SC Senate to reconvene in Sept. to discuss COV...,"[sc, senate, to, reconvene, in, sept, to, disc...","[sc, senate, to, reconvene, sept, to, discuss,...","[sc, senat, to, reconven, sept, to, discuss, c...","[sc, senate, to, reconvene, sept, to, discus, ...",sc senate to reconvene sept to discus covid19 ...
2,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,4.378954e+08,NaN,"Roseville, CA",during the covid19 crisis pres trump has not a...,"@JohnAvlon During the COVID-19 crisis, Pres. T...","[during, the, covid19, crisis, pres, trump, ha...","[the, covid19, crisis, pres, trump, always, ho...","[the, covid19, crisi, pre, trump, alway, hones...","[the, covid19, crisis, pres, trump, always, ho...",the covid19 crisis pres trump always honest se...
3,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,2.580088e+09,NaN,NaN,we just gonna act like wasn’t ahead of the cu...,We just gonna act like @JamesStormBrand wasn’t...,"[we, just, gon, na, act, like, wasn, ’, t, ahe...","[gon, na, act, like, ’, ahead, the, curve, sel...","[gon, na, act, like, ’, ahead, the, curv, sell...","[gon, na, act, like, ’, ahead, the, curve, sel...",gon na act like ahead the curve sell face mas...
4,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",the covid19 pandemic has created unprecedented...,The COVID-19 pandemic has created unprecedente...,"[the, covid19, pandemic, has, created, unprece...","[the, covid19, pandemic, created, unprecedente...","[the, covid19, pandem, creat, unpreced, barrie...","[the, covid19, pandemic, create, unprecedented...",the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...,...,...,...,...,...,...
7766,Tue Aug 18 19:34:12 +0000 2020,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",just so everyone can see what a republican man...,Just so everyone can see what a republican man...,"[just, so, everyone, can, see, what, a, republ...","[everyone, see, republican, man, sends, methey...","[everyon, see, republican, man, send, methey, ...","[everyone, see, republican, man, sends, methey...",everyone see republican man sends methey feel ...
7767,Tue Aug 18 19:34:13 +0000 2020,1.295797e+18,1.291778e+09,NaN,NaN,meanwhile canadians are constantly told they m...,"Meanwhile, Canadians are constantly told they ...","[meanwhile, canadians, are, constantly, told, ...","[meanwhile, canadians, constantly, told, must,...","[meanwhil, canadian, constantli, told, must, s...","[meanwhile, canadian, constantly, told, must, ...",meanwhile canadian constantly told must sacrif...
7768,Tue Aug 18 19:34:15 +0000 2020,1.295004e+18,3.486331e+08,NaN,Ontario,join us tuesday for an important backtoschool ...,Join us Tuesday for an important back-to-schoo...,"[join, us, tuesday, for, an, important, backto...","[join, us, tuesday, important, backtoschool, c...","[join, us, tuesday, import, backtoschool, chat...","[join, u, tuesday, important, backtoschool, ch...",join u tuesday important backtoschool chat the...
7769,Tue Aug 18 19:34:15 +0000 2020,1.295096e+18,2.666017e+09,NaN,Nottingham,due to the laughable recount of deaths in the ...,"Due to the laughable ""recount"" of deaths in th...","[due, to, the, laughable, recount, of, deaths,...

In [13]:
cols_2 = [0, 1, 2, 3, 4 ,6, 11]
data_final = cleaned_data[cleaned_data.columns.values[cols_2]]

In [14]:
data_final

,created_at,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,Tue Aug 18 19:30:11 +0000 2020,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...,...
7766,Tue Aug 18 19:34:12 +0000 2020,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,Tue Aug 18 19:34:13 +0000 2020,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,Tue Aug 18 19:34:15 +0000 2020,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,Tue Aug 18 19:34:15 +0000 2020,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...


In [15]:
data_final.to_csv('/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/(Sample_Cleaned&Analysed)h01-20200818-153021.csv',index=False)

In [18]:
START = time.time()
PATH = '/home/manikya_varshney/Documents/Python/Yale/h01-20200818-10files/final(Cleaned and Analysed)_h01-20200818-153021.csv'
DATA = pd.read_csv(PATH, index_col=None, header=0, engine='python' )
END = time.time()
print("Read parquet with pandas: ",(END-START),"sec")

Read parquet with pandas:  0.055869102478027344 sec


In [19]:
DATA

,id,user_id,coordinates,location,extended_tweet_full_text_duplicate,final
0,1.295805e+18,1.278716e+18,NaN,NaN,can we wear clone helmets instead of masks to ...,wear clone helmet instead mask to school
1,1.295805e+18,9.191072e+08,NaN,"Myrtle Beach, SC",SC Senate to reconvene in Sept. to discuss COV...,sc senate to reconvene sept to discus covid19 ...
2,1.295805e+18,4.378954e+08,NaN,"Roseville, CA","@JohnAvlon During the COVID-19 crisis, Pres. T...",the covid19 crisis pres trump always honest se...
3,1.295805e+18,2.580088e+09,NaN,NaN,We just gonna act like @JamesStormBrand wasn’t...,gon na act like ahead the curve sell face mas...
4,1.295805e+18,8.560174e+07,NaN,"Honolulu, Hawaii",The COVID-19 pandemic has created unprecedente...,the covid19 pandemic create unprecedented barr...
...,...,...,...,...,...,...
7766,1.295803e+18,7.455473e+17,NaN,"The Bay Area, California",Just so everyone can see what a republican man...,everyone see republican man sends methey feel ...
7767,1.295797e+18,1.291778e+09,NaN,NaN,"Meanwhile, Canadians are constantly told they ...",meanwhile canadian constantly told must sacrif...
7768,1.295004e+18,3.486331e+08,NaN,Ontario,Join us Tuesday for an important back-to-schoo...,join u tuesday important backtoschool chat the...
7769,1.295096e+18,2.666017e+09,NaN,Nottingham,"Due to the laughable ""recount"" of deaths in th...",due to the laughable recount death the uk the ...
